In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 8 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Méthodes de Runge Kutta : coût et précision
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#

# Méthodes de Runge Kutta : coût et précision

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
import warnings
warnings.filterwarnings('ignore')

## Modèle de Brusselator

La dynamique de la réaction oscillante découverte par Belousov et Zhabotinsky, peut être modélisée par le modèle dit de Brusselator en fonction de deux paramètres $a$ et $b$ :

$$
\left\{\begin{aligned}
d_t y_1 & = a - (b+1) y_1 + y_1^2y_2\\
d_t y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [ ]:
class brusselator_model:

    def __init__(self, a, b):
        self.a = a
        self.b = b

    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        y1_dot = a - (b+1)*y1 + y1*y1*y2
        y2_dot = b*y1 - y1*y1*y2
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        return np.array([[2*y1*y2 -(b+1), y1*y1], 
                         [-2*y1*y2 + b, -y1*y1] ])

### Solution quasi-exacte

La solution quasi-exacte est obtenue en utilisant une méthode de Runge-Kutta d’ordre 8 (DOP853) avec pas de temps adapatitifs.

In [ ]:
bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

tini = 0. 
tend = 20.

yini = (1.5, 3)

sol = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1.e-12, atol=1.e-12)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='y1'))
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name='y2'))
legend = dict(x=0.9, bgcolor='rgba(0,0,0,0)')
fig.update_layout(legend=legend, height=400, xaxis_title='t')

### Caractérisation de la raideur

In [ ]:
bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  
jac = bm.jac

tini = 0. 
tend = 20.

yini = (1.5 , 3)

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
for it, yi in enumerate(sol.y.transpose()):
    eig_vals[it] = np.linalg.eigvals(jac(0, yi))

lambda1 = eig_vals[:, 0]
lambda2 = eig_vals[:, 1]

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.2,
                    subplot_titles=("Partie réelle des valuers propres","Partie imaginaire des valeurs propres"))
color = fig.layout.template.layout.colorway
fig.add_trace(go.Scatter(x=sol.t, y=np.real(lambda1), name='\u03BB1', line_color=color[0], legendgroup='real'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=np.real(lambda2), name='\u03BB2', line_color=color[1], legendgroup='real'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=np.imag(lambda1), name='\u03BB1', line_color=color[0], legendgroup='imag'), row=2, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=np.imag(lambda2), name='\u03BB2', line_color=color[1], legendgroup='imag'), row=2, col=1)
legend = dict(x=0.9, bgcolor='rgba(0,0,0,0)', tracegroupgap=280, groupclick="toggleitem")
fig.update_layout(legend=legend, height=700)
fig.update_xaxes(title_text='t', row=1)
fig.update_xaxes(title_text='t', row=2)

## Méthodes de Runge-Kutta 

In [ ]:
def sol_and_error_bruss(yini, tini, tend, a, b, nt, method):
    
    if method == rk1:
        str_method = "RK1 (explicit Euler)"
    elif method == rk2:
        str_method = "RK2"
    elif method == rk3:
        str_method = "RK3"
    elif method == rk4:
        str_method = "RK4"
    else:
        print("Unkown method")
        return
    
    bm = brusselator_model(a, b)
    fcn = bm.fcn   

    nt_max= nt[-1]
    dt_max = (tend-tini)/(nt_max-1)
    
    sol = method(tini, tend, nt_max, yini, fcn)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", t_eval=sol.t, rtol=1.e-12, atol=1.e-12)
    err = np.abs(sol.y-sol_exa.y)

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Global error"), vertical_spacing=0.15)
    color = fig.layout.template.layout.colorway

    marker_y1 = dict(size=5, symbol='x-thin', line=dict(width=1, color=color[0]))
    line_y1 = dict(dash='dot', color=color[0])
    marker_y2 = dict(size=5, symbol='x-thin', line=dict(width=1, color=color[1]))
    line_y2 = dict(dash='dot', color=color[1])

    fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='y1', mode="lines+markers", legendgroup='sol',
                             marker=marker_y1, line=line_y1), row=1, col=1)
    fig.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name='y2', mode="lines+markers", legendgroup='sol',
                             marker=marker_y2, line=line_y2), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=sol.t, y=err[0], name='|y1exa-y1|', mode="markers", legendgroup='err',
                             marker=marker_y1), row=2, col=1)
    fig.add_trace(go.Scatter(x=sol.t, y=err[1], name='|y2exa-y1|', mode="markers", legendgroup='err',
                             marker=marker_y2), row=2, col=1)
    
    # create slider
    steps = []
    for i, nt_i in enumerate(nt):
        sol = method(tini, tend, nt_i, yini, fcn)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", t_eval=sol.t, rtol=1.e-12, atol=1.e-12)
        err = np.abs(sol.y-sol_exa.y)
        dt = (tend-tini)/(nt_i-1)
        step = dict(method="update", label = f"{nt_i}", args=[{"x": [sol.t, sol.t, sol.t, sol.t], 
                                                               "y": [sol.y[0], sol.y[1], err[0], err[1]]},
                            {"title": str_method+f": solution and error for dt={dt:.4e}"}])
        steps.append(step)

    sliders = [dict(active=len(nt)-1, currentvalue={"prefix": "nt : "}, steps=steps)]


    legend = dict(x=0.01, bgcolor='rgba(0,0,0,0)', tracegroupgap=320, groupclick="toggleitem")
    fig.update_layout(height=800, legend=legend, sliders=sliders, 
                      title=str_method+f": solution and error for dt={dt_max:.4e}")
    fig.update_yaxes(exponentformat='e', row=2)

    fig.show()

### Ordre 1

In [ ]:
class ode_result:
    def __init__(self, y, t, nfev):
        self.y = y
        self.t = t
        self.nfev = nfev 
        
def rk1(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)
        
    nfev = nt-1

    return ode_result(y, t, nfev)

In [ ]:
yini = (1.5 , 3)
tini = 0.
tend = 20.
a = 1.
b = 4.

nt_max = 2000
nt = np.hstack((np.array([233, 234]), np.geomspace(235, nt_max, num=10, dtype=int)))

sol_and_error_bruss(yini, tini, tend, a, b, nt, rk1)

### Ordre 2

In [ ]:
def rk2(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        y[:,it+1] = yn + dt*k2

    nfev = 2*(nt-1)

    return ode_result(y, t, nfev)

In [ ]:
yini = (1.5 , 3)
tini = 0.
tend = 20.
a = 1.
b = 4.

nt_max = 2000
nt = np.geomspace(205, nt_max, num=10, dtype=int)
nt = np.hstack((np.array([224, 225]), np.geomspace(226, nt_max, num=10, dtype=int)))

sol_and_error_bruss(yini, tini, tend, a, b, nt, rk2)

### Order 3

In [ ]:
def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + dt, yn + dt*(-k1 + 2*k2))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    nfev = 3*(nt-1)

    return ode_result(y, t, nfev)

In [ ]:
yini = (1.5 , 3)
tini = 0.
tend = 20.
a = 1.
b = 4.

nt_max = 2000
nt = np.hstack((np.array([130, 131, 132, 133, 134]), np.geomspace(135, nt_max, num=10, dtype=int)))

sol_and_error_bruss(yini, tini, tend, a, b, nt, rk3)

### Ordre 4

In [ ]:
def rk4(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + 0.5*dt, yn + dt*(0.5*k2))
        k4 = fcn(tn + dt, yn + dt*k3)
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

In [ ]:
yini = (1.5 , 3)
tini = 0.
tend = 20.
a = 1.
b = 4.

nt_max = 2000
nt = np.hstack((np.array([130, 131, 132, 133, 134]), np.geomspace(135, nt_max, num=10, dtype=int)))

sol_and_error_bruss(yini, tini, tend, a, b, nt, rk4)

### Coût et précision

In [ ]:
tini = 0.
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = np.array([1001, 2001, 5001, 10001, 25001, 50001, 100001, 200001])

fe_rk1=[]; norm_rk1=[]
fe_rk2=[]; norm_rk2=[]
fe_rk3=[]; norm_rk3=[]
fe_rk4=[]; norm_rk4=[]

for nt_i in nt:

    t = np.linspace(tini,tend,nt_i)
    
    rtol=1e-14; atol=1e-14
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=rtol, atol=atol, t_eval=t)

    sol_rk1 = rk1(tini, tend, nt_i, yini, fcn)
    fe_rk1.append(sol_rk1.nfev)
    norm_rk1.append(np.linalg.norm(sol_exa.y-sol_rk1.y) / np.sqrt(nt_i))

    sol_rk2 = rk2(tini, tend, nt_i, yini, fcn)
    fe_rk2.append(sol_rk2.nfev)
    norm_rk2.append(np.linalg.norm(sol_exa.y-sol_rk2.y) / np.sqrt(nt_i))
    
    sol_rk3 = rk3(tini, tend, nt_i, yini, fcn)
    fe_rk3.append(sol_rk3.nfev)
    norm_rk3.append(np.linalg.norm(sol_exa.y-sol_rk3.y) / np.sqrt(nt_i))
    
    sol_rk4 = rk4(tini, tend, nt_i, yini, fcn)
    fe_rk4.append(sol_rk4.nfev)
    norm_rk4.append(np.linalg.norm(sol_exa.y-sol_rk4.y) / np.sqrt(nt_i))

dt = (tend-tini)/(nt-1)    
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=fe_rk1, y=norm_rk1, name='rk1', marker_symbol='x'))
fig.add_trace(go.Scatter(x=fe_rk2, y=norm_rk2, name='rk2', marker_symbol='x'))
fig.add_trace(go.Scatter(x=fe_rk3, y=norm_rk3, name='rk3', marker_symbol='x'))
fig.add_trace(go.Scatter(x=fe_rk4, y=norm_rk4, name='rk4', marker_symbol='x'))
fig.update_xaxes(type='log', exponentformat='e', title="Nombre d'évaluations de fonction")
fig.update_yaxes(type='log', exponentformat='e', title="Norme l2 de l'erreur")
fig.update_layout(height=500, legend=dict(x=0.9, bgcolor='rgba(0,0,0,0)'))